In [1]:
from utils.dataloader import *
from utils.sift import *
from utils.helper import *
from utils.classifier import get_report, down_sampling, vote, up_sampling
import pickle


In [2]:
RANDOME_STATE= 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
train_collection = test_collection = y_train = y_test = None


with open(os.path.join(OUT_PATH, 'sift_X.pkl'), 'rb') as f:
    (train_collection, test_collection) = pickle.load(f)


with open(os.path.join(OUT_PATH, 'sift_y.pkl'), 'rb') as f:
    (y_train, y_test) = pickle.load(f)


In [4]:

ks = train_collection.keys()
y_train = np.array(y_train)
y_test = np.array(y_test)

for a, b in zip(train_collection.values(), test_collection.values()):
    print(a.shape, b.shape)

print(ks)
print(y_train.shape, y_test.shape)

(1968, 128) (656, 128)
(1968, 256) (656, 256)
(1968, 512) (656, 512)
(1968, 1024) (656, 1024)
dict_keys([128, 256, 512, 1024])
(1968,) (656,)


# Undersampling method

441
88
42
235
690
133
37
302

In [5]:
from sklearn.svm import SVC

param = {
    'kernel': ['linear', 'rbf'],
    'C' : [0.01, 0.1, 1, 10, 100]
}


clf_collections = []
predictions_collections = []
votes_collections = []


In [6]:

logger = my_logger(os.path.join(OUT_PATH, 'logs/undersampling.log'), 'undersampling')
logger.warning(f'k,i,j,best_score,,best_clf,mean_train_score,mean_test_score')

total_classes = np.unique(y_train)

for k in ks:
    
    X_train = np.array(train_collection[k])
    X_test = np.array(test_collection[k])
    clfs = []
    predictions = []
    
    clfs = down_sampling(total_classes=total_classes, X_train=X_train, y_train=y_train, 
                         clf = SVC, clfs = clfs, 
                         param=param, k=k, logger=logger)


    predictions, votes, counts = vote(X_test=X_test, clfs=clfs, predictions=predictions)
    
    acc, conf_mat = get_report(y_test, votes)
    print()
    print(f'Overall Test Accuracy: {acc:.2f}')
    print(f'Confusion Matrix: \n{conf_mat}')
    print('-'*100)
    
    predictions_collections.append(predictions)
    votes_collections.append(votes)
    clf_collections.append(clfs)
    
    logger.handlers[0].flush()

            

Class: [0 1] Val Score: 0.74 use {'C': 0.1, 'kernel': 'linear'}
Class: [0 2] Val Score: 0.65 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 3] Val Score: 0.86 use {'C': 10, 'kernel': 'rbf'}
Class: [0 4] Val Score: 1.00 use {'C': 10, 'kernel': 'rbf'}
Class: [0 5] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 6] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 7] Val Score: 1.00 use {'C': 100, 'kernel': 'rbf'}
Class: [1 2] Val Score: 0.73 use {'C': 0.1, 'kernel': 'rbf'}
Class: [1 3] Val Score: 0.85 use {'C': 10, 'kernel': 'rbf'}
Class: [1 4] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Class: [1 5] Val Score: 0.99 use {'C': 0.1, 'kernel': 'rbf'}
Class: [1 6] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Class: [1 7] Val Score: 0.99 use {'C': 0.1, 'kernel': 'rbf'}
Class: [2 3] Val Score: 0.78 use {'C': 0.01, 'kernel': 'linear'}
Class: [2 4] Val Score: 0.99 use {'C': 1, 'kernel': 'rbf'}
Class: [2 5] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [2

# SMOTE Upsampling

In [7]:
from sklearn.neighbors import KNeighborsClassifier


logger = my_logger(os.path.join(OUT_PATH, 'logs/upsampling.log'), 'upsampling')
logger.warning(f'k,i,j,best_score,,best_clf,mean_train_score,mean_test_score')


total_classes = np.unique(y_train)
for k in ks:
    
    X_train = np.array(train_collection[k])
    X_test = np.array(test_collection[k])
    clfs = []
    predictions = []
    
    clfs = up_sampling(total_classes=total_classes, X_train=X_train, y_train=y_train, 
                         clf = SVC, clfs = clfs, 
                         param=param, k=k, logger=logger,
                         knn = KNeighborsClassifier)


    predictions, votes, counts = vote(X_test=X_test, clfs=clfs, predictions=predictions)
    
    acc, conf_mat = get_report(y_test, votes)
    print()
    print(f'Overall Test Accuracy: {acc:.2f}')
    print(f'Confusion Matrix: \n{conf_mat}')
    print('-'*100)
    
    predictions_collections.append(predictions)
    votes_collections.append(votes)
    clf_collections.append(clfs)
    
    logger.handlers[0].flush()

            

Generated 440 synthetic samples.
Class: [0. 1.] Val Score: 0.92 use {'C': 100, 'kernel': 'rbf'}
Generated 420 synthetic samples.
Class: [0. 2.] Val Score: 0.94 use {'C': 100, 'kernel': 'rbf'}
Generated 235 synthetic samples.
Class: [0. 3.] Val Score: 0.92 use {'C': 100, 'kernel': 'rbf'}
Generated 441 synthetic samples.
Class: [0. 4.] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Generated 399 synthetic samples.
Class: [0. 5.] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Generated 407 synthetic samples.
Class: [0. 6.] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Generated 302 synthetic samples.
Class: [0. 7.] Val Score: 1.00 use {'C': 10, 'kernel': 'rbf'}
Generated 84 synthetic samples.
Class: [1. 2.] Val Score: 0.85 use {'C': 10, 'kernel': 'linear'}
Generated 176 synthetic samples.
Class: [1. 3.] Val Score: 0.93 use {'C': 100, 'kernel': 'rbf'}
Generated 616 synthetic samples.
Class: [1. 4.] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Generated 88 synthetic samples.
Clas